# DERIVING SPATIAL MAPS WITH PYFLWDR
## Tools with GDAL

### TOOL: PROJWIN extend of window

In [1]:
""" [-projwin <ulx> <uly> <lrx> <lry>]
		Xmin Ymin Xmax Ymax"""

def PROJWINDOW(Coord_UperLeft, Coord_LowerRight):
	Flag_SouthHemisphere = True
	
	Xmin = min(Coord_UperLeft[1], Coord_LowerRight[1])
	Ymin = min(Coord_UperLeft[0], Coord_LowerRight[0])
	Xmax = max(Coord_UperLeft[1], Coord_LowerRight[1])
	Ymax = max(Coord_UperLeft[0], Coord_LowerRight[0])

	if Flag_SouthHemisphere:
		Projwindow = str(Xmin) + ' ' + str(Ymax) + ' ' + str(Xmax) + ' ' + str(Ymin)
	else:
		Projwindow = str(Xmin) + ' ' + str(Ymin) + ' ' + str(Xmax) + ' ' + str(Ymax)

	print(Projwindow)
	return Projwindow


### TOOLS: Converting Vector to raster *.tiff*


In [2]:
import subprocess as subprocess
import os as os

# -ot {Byte/Int8/Int16/UInt16/UInt32/Int32/UInt64/Int64/Float32/Float64/ CInt16/CInt32/CFloat32/CFloat64}]

def VECTOR_2_TIFF(Name_Input, Path_Input, Name_Output, Path_Output, Coord_DEM_UL, Coord_DEM_LR, Cell_Size, Attribute_name, Projection, Type, Option_Crop, NoDataValue):

	print ('\n Starting converting vector to raster: ' + Name_Output)

	os.chdir(Path_Output)
	Name_Input_NoPath = os.path.splitext(Name_Input)[0]

	# TRANSFORMING VECTOR to RASTER-----------------------------------------------------------------------------------------------------
	if Option_Crop:
		Projwin = PROJWINDOW(Coord_DEM_UL, Coord_DEM_LR)

		# DosCommand = 'gdal_rasterize ' + ' -init 0 ' + ' -l ' + Name_Input_NoPath + ' -a ' + Attribute_name + ' -tr ' + str(Cell_Size) + ' ' + str(Cell_Size) + ' -a_srs ' + Projection  + ' -te ' + Projwin + ' -a_nodata '+ str(NoDataValue) + ' -ot ' + Type + ' -of GTiff ' + Path_Input + '/' + Name_Input + ' ' + Path_Output + '/' + Name_Output

		DosCommand = 'gdal_rasterize ' + ' -a_srs ' + Projection + ' -init 0 ' + ' -l ' + Name_Input_NoPath + ' -a ' + Attribute_name + ' -tr ' + str(Cell_Size) + ' ' + str(Cell_Size)  + ' -te ' + Projwin + ' -a_nodata '+ str(NoDataValue) + ' -ot ' + Type + ' -of GTiff ' + Path_Input + '/' + Name_Input + ' ' + Path_Output + '/' + Name_Output
	
	else:
		# DosCommand = 'gdal_rasterize ' + ' -l ' + Name_Input_NoPath + ' -a ' + Attribute_name + ' -tr ' + str(Cell_Size) + ' ' + str(Cell_Size) + ' -a_srs ' + Projection  + ' -a_nodata ' + str(NoDataValue)  + ' -ot ' + Type + ' -of GTiff ' + Path_Input + '/' + Name_Input + ' ' + Path_Output + '/' + Name_Output

		DosCommand = 'gdal_rasterize ' + ' -a_srs ' + Projection + ' -l ' + Name_Input_NoPath + ' -a ' + Attribute_name + ' -tr ' + str(Cell_Size) + ' ' + str(Cell_Size)  + ' -a_nodata ' + str(NoDataValue)  + ' -ot ' + Type + ' -of GTiff ' + Path_Input + '/' + Name_Input + ' ' + Path_Output + '/' + Name_Output

	print(" ")
	print(DosCommand)
	print(" ")

	subprocess.check_output(DosCommand).decode()

	print ('Sucessfull converting vector to raster: ' + Name_Output)
	print(" ")


### TOOLS: resampling method with new coordinate and raster size

In [3]:

""" -r resampling_method:
	Resampling method to use. Available methods are:
	near: nearest neighbour resampling (default, fastest algorithm, worst interpolation quality).
	bilinear: bilinear resampling.
	cubic: cubic resampling.
	cubicspline:cubic spline resampling.
	lanczos: Lanczos windowed sinc resampling.
	average: average resampling, computes the average of all non-NODATA contributing pixels. (GDAL >= 1.10.0)
	mode: mode resampling, selects the value which appears most often of all the sampled points. (GDAL >= 1.10.0)
	max: maximum resampling, selects the maximum value from all non-NODATA contributing pixels. (GDAL >= 2.0.0)
	min: minimum resampling, selects the minimum value from all non-NODATA contributing pixels. (GDAL >= 2.0.0)
	med: median resampling, selects the median value of all non-NODATA contributing pixels. (GDAL >= 2.0.0)
	q1: first quartile resampling, selects the first quartile value of all non-NODATA contributing pixels. (GDAL >= 2.0.0)
	q3:# third quartile resampling, selects the third quartile value of all non-NODATA contributing pixels. (GDAL >= 2.0.0)"""

import subprocess as subprocess
import os as os

def REPROJECTING_RESAMPLING(Name_Input, Path_Input, Name_Output, Path_Output, Coord_DEM_UL, Coord_DEM_LR, CellSize_X, CellSize_Y, Projection, resampling_method):

	Projwin = PROJWINDOW(Coord_DEM_UL, Coord_DEM_LR)

	print('\n STARTING REPROJECTING_RESAMPLING: ' + Name_Output)

	DosCommand = "gdalwarp -overwrite -of gtiff -t_srs " + Projection + " -te " + Projwin + " -tr " + str(CellSize_X) + ' ' + str(CellSize_Y) + " -tap -r " + resampling_method + " " + Path_Input + '/' + Name_Input + ' ' + Path_Output + '/' + Name_Output

	print(" ")
	print(DosCommand)
	print(" ")

	subprocess.check_output(DosCommand, shell=True).decode()

	print('\n SUCESSFULL REPROJECTING_RESAMPLING: ' + Name_Output)


### TOOLS: Bounding boxes

In [4]:
def BOUNDINGBOX(Coord_TopLeft, Coord_BottomRight):
	# Coord_TopLeft     = [Coord_Y_Top, Coord_X_Left]
	# Coord_BottomRight = [Coord_Y_Bottom, Coord_X_Right]

	Coord_X_Left   = Coord_TopLeft[1]
	Coord_Y_Bottom = Coord_BottomRight[0]
	Coord_X_Right  = Coord_BottomRight[1]
	Coord_Y_Top    = Coord_TopLeft[0]

	BoundingBox = [Coord_X_Left, Coord_Y_Bottom, Coord_X_Right, Coord_Y_Top]

	return BoundingBox


## RASTERIO

### TOOLS: info

In [5]:
import rasterio

def INFORMATION(Path_Input, Verbose=True):

	Rasterio = rasterio.open(Path_Input)
	Dimensions   = Rasterio.transform

	CellSize_X   = Dimensions[0]
	CellSize_Y   = Dimensions[4]
	N_Width      = Rasterio.width
	N_Hight      = Rasterio.height
	CRS          = Rasterio.crs
	Coord_X_Left   = Rasterio.bounds[0]
	Coord_X_Right  = Rasterio.bounds[2]
	Coord_Y_Top    = Rasterio.bounds[3]
	Coord_Y_Bottom = Rasterio.bounds[1]

	Coord_TopLeft     = [Coord_Y_Top, Coord_X_Left]
	Coord_BottomRight = [Coord_Y_Bottom, Coord_X_Right]

	if Verbose:
		print("Cell size_X=", CellSize_X )
		print("Cell size_Y=", CellSize_Y )
		print("DEM width= " , N_Width) 
		print("DEM height= ", N_Hight)
		print("Coordinate Reference System CRS=" , CRS)
		print("Bounds= ", Rasterio.bounds)
		print("Bound_Left= ", Coord_X_Left ," ,Bound_Right= ", Coord_X_Right ," ,Bound_Top= ", Coord_Y_Top ," ,Bound_Botton= ", Coord_Y_Bottom)
		print("Coord_TopLeft= ", Coord_TopLeft)
		print("Coord_BottomRight= ", Coord_BottomRight)

	return CellSize_X, CellSize_Y, Coord_BottomRight, Coord_TopLeft, Coord_X_Left, Coord_X_Right, Coord_Y_Bottom, Coord_Y_Top, CRS, N_Hight, N_Width, Rasterio


## TOOLS PYFLWDIR

### TOOLS: From DEM -> FlowDirection

In [6]:
import rasterio
import numpy as np
import os as os
import pyflwdir

def DEM_2_FLOWDIRECTION(Name_Input, Path_Input, Name_Output, Path_Output):
   
	Path = os.path.join(Path_Input, Name_Input)

	# Read elevation data using rasterio
	with rasterio.open(Path, "r") as src:
		elevtn = src.read(1)
		nodata = src.nodata
		transform = src.transform
		crs = src.crs
		#  extent = np.array(src.bounds)[[0, 2, 1, 3]]
		latlon = src.crs.is_geographic
		prof = src.profile

	# returns FlwDirRaster object
	FlowDirection_Pyflwdir = pyflwdir.from_dem(data=elevtn, nodata=src.nodata, transform=transform, latlon=latlon, outlets="min")

	FlowDirection_Array = FlowDirection_Pyflwdir.to_array(ftype="ldd")

	# Write to tiff file
	Path = os.path.join(Path_Output, Name_Output)
	prof.update(dtype=FlowDirection_Array.dtype, nodata=False)
	with rasterio.open(Path, "w", **prof) as src:
		src.write(FlowDirection_Array, 1)

	return FlowDirection_Array, FlowDirection_Pyflwdir


### TOOLS: From DEM -> SLOPE

In [7]:
import rasterio
import pyflwdir
import os as os

def DEM_2_SLOPE(Name_Input_Dem, Path_Input_Dem, Name_Output_Slope, Path_Output_Slope):
	
	Path = os.path.join(Path_Input_Dem, Name_Input_Dem)
 
 	# Read elevation data using rasterio
	with rasterio.open(Path, "r") as src:
		elevtn = src.read(1)
		nodata = src.nodata
		transform = src.transform
		crs = src.crs
		#  extent = np.array(src.bounds)[[0, 2, 1, 3]]
		latlon = src.crs.is_geographic
		prof = src.profile
 
	Slope_Pyflwdir = pyflwdir.dem.slope(elevtn, nodata=src.nodata, latlon=latlon, transform=transform)
 
 	# Write to tiff file
	Path = os.path.join(Path_Output_Slope, Name_Output_Slope)
 
	prof.update(dtype=Slope_Pyflwdir.dtype, nodata=src.nodata)
	with rasterio.open(Path, "w", **prof) as src:
		src.write(Slope_Pyflwdir, 1)
 
	return Slope_Pyflwdir



### TOOLS: From FlowDirection -> Streams

In [8]:
import rasterio
import pyflwdir
import numpy as np
import os as os

def FLOWDIRECTION_2_STREAMS(Name_Input_Dem, Path_Input_Dem, Name_Output_Stream, Name_Output_StreamOrder, Name_Output_StreamWidth, Name_Output_StreamSlope, Path_Output_Stream, FlowDirection_Pyflwdir, Slope_Pyflwdir, StreamOrder=4, StreamWidth=1.0, Type='strahler'):
   
   #  Type= "strahler" or "classic"
   
   Path_Dem = os.path.join(Path_Input_Dem, Name_Input_Dem)
   
   # Read elevation data using rasterio   
   with rasterio.open(Path_Dem, "r") as src:
      elevtn = src.read(1)
      nodata = src.nodata
      transform = src.transform
      crs = src.crs
      # extent = np.array(src.bounds)[[0, 2, 1, 3]]
      latlon = src.crs.is_geographic
      prof = src.profile
   
   # === STREAM ORDERS ====
   Streams_StreamOrder = FlowDirection_Pyflwdir.stream_order(type=Type)
   
   # Masking data were Stream order > 1
   Nx,Ny = np.shape(Streams_StreamOrder)
   Streams_StreamOrder2 = np.ma.empty(shape=(Nx, Ny))
   for iX in range(Nx):
      for iY in range(Ny):
         if Streams_StreamOrder[iX,iY] > 1:
            Streams_StreamOrder2[iX,iY] = Streams_StreamOrder[iX,iY]
   
   Path = os.path.join(Path_Output_Stream, Name_Output_StreamOrder)
   
   prof.update(dtype=Streams_StreamOrder.dtype, nodata=False)
   with rasterio.open(Path, "w", **prof) as src:
      src.write(Streams_StreamOrder2, 1)
      
   # === STREAMS TRUE ===
   Path = os.path.join(Path_Output_Stream, Name_Output_Stream)
   
   Streams = np.greater(Streams_StreamOrder, StreamOrder)
   
   prof.update(dtype=Streams_StreamOrder.dtype, nodata=False)
   with rasterio.open(Path, "w", **prof) as src:
      src.write(Streams, 1)
   
   # === STREAMS WIDTH ===
   Path = os.path.join(Path_Output_Stream, Name_Output_StreamWidth)
     
   Streams_Width = Streams * StreamWidth
   
   prof.update(dtype=Streams_StreamOrder.dtype, nodata=False)
   with rasterio.open(Path, "w", **prof) as src:
      src.write(Streams_Width, 1)
      
	# === STREAM SLOPE ====
   Path = os.path.join(Path_Output_Stream, Name_Output_StreamSlope)
   
   Nx,Ny = np.shape(Streams)
   Streams_Slope = np.ma.empty(shape=(Nx, Ny))
   
   # Masking onl data were we have slope
   for iX in range(Nx):
      for iY in range(Ny):
         if Streams[iX,iY]==True:
            Streams_Slope[iX,iY] = Slope_Pyflwdir[iX,iY]

   
   prof.update(dtype=Slope_Pyflwdir.dtype, nodata=False)
   with rasterio.open(Path, "w", **prof) as src:
      src.write(Streams_Slope, 1) 
      
   return Streams, Streams_Slope, Streams_StreamOrder, Streams_Width
   

### TOOLS FlowDirections -> Catchements

In [9]:
# import geopandas as gpd
import numpy as np
import rasterio
import pyflwdir
# import python_utils
import os as os

def FLOWDIRECTION_2_CATCHEMENT(FlowDirection_Pyflwdir, Name_Input_Dem, Path_Input_Dem, Name_Output, Path_Output, Option_Catchement="Subbasins"):

   Path_Dem = os.path.join(Path_Input_Dem, Name_Input_Dem)

   
   # Read elevation data using rasterio   
   with rasterio.open(Path_Dem, "r") as src:
      elevtn = src.read(1)
      nodata = src.nodata
      transform = src.transform
      crs = src.crs
      # extent = np.array(src.bounds)[[0, 2, 1, 3]]
      latlon = src.crs.is_geographic
      prof = src.profile
   
   # Delinating catchements
   if  Option_Catchement == "Subbasins":
      Subbassins, Idxs_out = FlowDirection_Pyflwdir.subbasins_streamorder(min_sto=7, mask=None)
   
   Path = os.path.join(Path_Output, Name_Output)
   
   prof.update(dtype=Subbassins.dtype, nodata=False)
   with rasterio.open(Path, "w", **prof) as src:
      src.write(Subbassins, 1) 
   
   return Subbassins, Idxs_out


# =========================================================================================================

# DATA


In [10]:
import os

Projection_NZ        = 'EPSG:2193'  # This is the default projection NZGD2000 / New Zealand Transverse Mercator 2000
Input_Static_Root    = "D:\MAIN\MODELS\WFLOW\DATA\SouthLand\Input\Static"
Dem_Name_Tif         = "DEM.tif"

Root_Output_Tiff     = "D:\MAIN\MODELS\WFLOW\DATA\SouthLand\OutputTiff"
Root_Output_Csv      = "D:\MAIN\MODELS\WFLOW\DATA\SouthLand\OutputCsv"

Coord_DEM_UperLeft   = [4906000, 1245910]
Coord_DEM_LowerRight = [4849880, 1310500]

CellSize_X = 8.0
CellSize_Y = 8.0

StreamWidth = 1.0

Path_Input_Dem = os.path.join(Input_Static_Root, Dem_Name_Tif)

Option_Subcatchement_OR_Catchement = 'Subcatchement' # 'Subcatchement' 'Catchement'

StreamOrder = 4


## DEM TASKS

In [11]:
#DEM resampling

CellSize_X, CellSize_Y, Coord_BottomRight, Coord_TopLeft, Coord_X_Left, Coord_X_Right, Coord_Y_Bottom, Coord_Y_Top, CRS, N_Hight, N_Width, Rasterio_Dem = INFORMATION(Path_Input_Dem, Verbose=False)

REPROJECTING_RESAMPLING("DEM.tif", Input_Static_Root, "DEM_Resampled.tif", Root_Output_Tiff, Coord_DEM_UperLeft, Coord_DEM_LowerRight, CellSize_X, CellSize_Y, Projection_NZ, "average")

Path_DemResampled = os.path.join(Root_Output_Tiff, "DEM_Resampled.tif")

CellSize_X, CellSize_Y, Coord_BottomRight, Coord_TopLeft, Coord_X_Left, Coord_X_Right, Coord_Y_Bottom, Coord_Y_Top, CRS, N_Hight, N_Width, Rasterio_Dem = INFORMATION(Path_DemResampled)


RasterioIOError: D:\MAIN\MODELS\WFLOW\DATA\SouthLand\Input\Static\DEM.tif: No such file or directory

### Flow directions

In [ ]:
FlowDirection_Array, FlowDirection_Pyflwdir = DEM_2_FLOWDIRECTION("DEM_Resampled.tif", Root_Output_Tiff, "FlowDirection.tiff", Root_Output_Tiff)


### Slope 

In [ ]:
Slope_Pyflwdir = DEM_2_SLOPE("DEM_Resampled.tif", Root_Output_Tiff, "Slope.tiff", Root_Output_Tiff)


### Streams

In [ ]:
Streams, Streams_Slope, Streams_StreamOrder, Streams_Width = FLOWDIRECTION_2_STREAMS("DEM_Resampled.tif", Root_Output_Tiff, "Streams.tiff", "StreamsOrder.tiff", "StreamsWidth.tiff", "StreamSlope.tiff", Root_Output_Tiff, FlowDirection_Pyflwdir, Slope_Pyflwdir, StreamOrder, StreamWidth, Type='strahler')


### Catchement boundary

In [ ]:
Subbassins, Idxs_out = FLOWDIRECTION_2_CATCHEMENT(FlowDirection_Pyflwdir, "DEM_Resampled.tif", Root_Output_Tiff, "Catchment.tiff", Root_Output_Tiff, Option_Catchement="Subbasins")


#### DEM data: *wflow_dem*


### River network

#### Gauges stations: *wflow_gauges_grdc*

#### ldd maps: *wflow_ldd*

### Looking for pits

#### Subcatchement: *wflow_subcatch*

### Reduce extend based on catchment


In [ ]:

Coord_DEM_UperLeft, Coord_DEM_LowerRight, Cell_X, Cell_Y = MAP_EXTENT(CatchementMap, Cell_Size)


### River network

##### Slope: *Slope*

### MAPS derived from rivers

#### River location: *wflow_river*

#### River length: *wflow_riverlength*

#### River width: *wflow_riverwidth*


#### River slope: *RiverSlope*

## OUTPUT CSV

## NetCDF Static data

## FORCING DATA

### Precipitation


#### Potential evapotranspiration

In [ ]:
# !mamba create -n WflowDiscretisation
# !conda activate WflowDiscretisation
# !conda install rasterio





 
